# HU Extension                     Assignment 04           E63 Big Data Analytics
## Issued on: Feb 17, 2023                        Due on Saturday by 11:59 AM EST, Feb 25, 2023
## Emmanuel Aboah

### Problem 1. 
Expand provided archive test_db-master.zip. You will see file employees.sql which you could use to create MySQL database employees. In the directory where that file resides run:
mysql --user=root --password < employees.sql
Provide the password and the database will be created and loaded.
Using MySQL prompt, find the total number of departments and the number of employees in each department. 
Your second result should include names of the departments.
From table dept_emp, delete all rows representing the department with the smallest number of employees.
Into table departments insert new department named “Entertainment”. 
Into table employees insert one new employee named “John Smith”.

Create schema and load data

```bash
mysql --user=root --password < employees.sql

# Output:
Enter password: 
INFO
CREATING DATABASE STRUCTURE
INFO
storage engine: InnoDB
INFO
LOADING departments
INFO
LOADING employees
INFO
LOADING dept_emp
INFO
LOADING dept_manager
INFO
LOADING titles
INFO
LOADING salaries
data_load_time_diff
00:01:23
```

Find the total number of departments and the number of employees in each department.

```bash
# Total number of departments.
mysql> select count(*) from departments;
+----------+
| count(*) |
+----------+
|        9 |
+----------+
1 row in set (0.00 sec)

# Number of employees in each department.
mysql> select de.dept_no, d.dept_name, count(de.emp_no) as total_emps 
    -> from dept_emp de
    -> join departments d on de.dept_no = d.dept_no
    -> group by dept_no;
+---------+--------------------+------------+
| dept_no | dept_name          | total_emps |
+---------+--------------------+------------+
| d005    | Development        |      85707 |
| d007    | Sales              |      52245 |
| d004    | Production         |      73485 |
| d003    | Human Resources    |      17786 |
| d008    | Research           |      21126 |
| d006    | Quality Management |      20117 |
| d009    | Customer Service   |      23580 |
| d001    | Marketing          |      20211 |
| d002    | Finance            |      17346 |
+---------+--------------------+------------+

# Department with least number of employees.
select de.dept_no, d.dept_name, count(de.emp_no) as total_emps 
    -> from dept_emp de
    -> join departments d on de.dept_no = d.dept_no
    -> group by dept_no
    -> order by total_emps asc limit 1;
+---------+-----------+------------+
| dept_no | dept_name | total_emps |
+---------+-----------+------------+
| d002    | Finance   |      17346 |
+---------+-----------+------------+
1 row in set (0.79 sec)

# From table dept_emp, Delete all rows representing the department with the smallest number of employees.
mysql> delete from dept_emp where dept_no = 'd002';
17346 row(s) affected	0.719 sec
# Proof:
mysql> select * from dept_emp where dept_no = 'd002';
Empty set (0.23 sec)

# Into table departments insert new department named “Entertainment”. 
mysql> insert into departments (dept_no, dept_name) values('d010', 'Entertainment');
Query OK, 1 row affected (0.01 sec)
# Proof
mysql> select * from departments where dept_no = 'd010';
+---------+---------------+
| dept_no | dept_name     |
+---------+---------------+
| d010    | Entertainment |
+---------+---------------+
1 row in set (0.00 sec)

# Into table employees insert one new employee named “John Smith”.
INSERT INTO employees (emp_no, birth_date, first_name, last_name, gender, hire_date) VALUES ('500000', '1990-10-10', 'John', 'Smith', 'M', '2019-10-10');
Query OK, 1 row affected (0.03 sec)
# Proof:
mysql> select * from employees where emp_no = 500000;
+--------+------------+------------+-----------+--------+------------+
| emp_no | birth_date | first_name | last_name | gender | hire_date  |
+--------+------------+------------+-----------+--------+------------+
| 500000 | 1990-10-10 | John       | Smith     | M      | 2019-10-10 |
+--------+------------+------------+-----------+--------+------------+
1 row in set (0.00 sec)
```

### Problem 2. 
On the Spark side, create Data Frames df_employees, df_dept_emp and df_departments and populate them with the contents of tables employees, dept_emp and departments from the above database. 
Using DataFrame API or spark.SQL statements create to new DataFrames: employee (name in singular) and department (name in singular). 
DataFrame employee will contain all the rows and columns of df_employees plus properly populated dept_no column. 
DataFrame department will contain all the columns of the DataFrame df_departments. Using those two new DataFrames, create and populate corresponding table in MySQL database employee and department. Report on the number of rows in each MySQL table. 

In [104]:
# Imports
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import findspark

findspark.init()

In [ ]:
# Set context and configs
conf = SparkConf().setMaster("local").setAppName("Hw04App")

sc = SparkContext(conf=conf)

sc.setLogLevel("ERROR")

In [ ]:
# SparkSession
spark = SparkSession(sc)

Using spark, create Data Frames df_employees, df_dept_emp and df_departments and populate them with the contents of tables employees, dept_emp and departments from the above database

In [109]:
# employees
df_employees = (
    spark.read
    .format("jdbc")
    .option("url","jdbc:mysql://127.0.0.1:3306")
    .option("driver","com.mysql.cj.jdbc.Driver")
    .option("dbtable", "employees.employees")
    .option("user", "root")
    .option("password", "password")
    .load()
)

df_employees.printSchema()

df_employees.show(5)

root
 |-- emp_no: integer (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- hire_date: date (nullable = true)



+------+----------+----------+---------+------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|
+------+----------+----------+---------+------+----------+
| 10001|1953-09-02|    Georgi|  Facello|     M|1986-06-26|
| 10002|1964-06-02|   Bezalel|   Simmel|     F|1985-11-21|
| 10003|1959-12-03|     Parto|  Bamford|     M|1986-08-28|
| 10004|1954-05-01| Chirstian|  Koblick|     M|1986-12-01|
| 10005|1955-01-21|   Kyoichi| Maliniak|     M|1989-09-12|
+------+----------+----------+---------+------+----------+
only showing top 5 rows



In [ ]:
# dept_emp
df_dept_emp = (
    spark.read
    .format("jdbc")
    .option("url","jdbc:mysql://127.0.0.1:3306")
    .option("driver","com.mysql.cj.jdbc.Driver")
    .option("dbtable", "employees.dept_emp")
    .option("user", "root")
    .option("password", "password")
    .load()
)

df_dept_emp.printSchema()

df_dept_emp.show(5)

root
 |-- emp_no: integer (nullable = true)
 |-- dept_no: string (nullable = true)
 |-- from_date: date (nullable = true)
 |-- to_date: date (nullable = true)



+------+-------+----------+----------+
|emp_no|dept_no| from_date|   to_date|
+------+-------+----------+----------+
| 10001|   d005|1986-06-26|9999-01-01|
| 10002|   d007|1996-08-03|9999-01-01|
| 10003|   d004|1995-12-03|9999-01-01|
| 10004|   d004|1986-12-01|9999-01-01|
| 10005|   d003|1989-09-12|9999-01-01|
+------+-------+----------+----------+
only showing top 5 rows



In [ ]:
# departments
df_departments = (
    spark.read
    .format("jdbc")
    .option("url","jdbc:mysql://127.0.0.1:3306")
    .option("driver","com.mysql.cj.jdbc.Driver")
    .option("dbtable", "employees.departments")
    .option("user", "root")
    .option("password", "password")
    .load()
)

df_departments.printSchema()

df_departments.show()

root
 |-- dept_no: string (nullable = true)
 |-- dept_name: string (nullable = true)

+-------+------------------+
|dept_no|         dept_name|
+-------+------------------+
|   d009|  Customer Service|
|   d005|       Development|
|   d010|     Entertainment|
|   d002|           Finance|
|   d003|   Human Resources|
|   d001|         Marketing|
|   d004|        Production|
|   d006|Quality Management|
|   d008|          Research|
|   d007|             Sales|
+-------+------------------+



Using DataFrame API or spark.SQL statements create two new DataFrames: employee (name in singular) and department (name in singular). 
DataFrame employee will contain all the rows and columns of df_employees plus properly populated dept_no column. DataFrame department will contain all the columns of the DataFrame df_departments.

In [ ]:
# Employee DF
df_employee = (
    df_employees
    .join(df_dept_emp, ["emp_no"])
    .join(df_departments, ["dept_no"])
    .select(df_employees["*"], df_dept_emp["dept_no"])
    )

df_employee.show(5)

+------+----------+----------+---------+------+----------+-------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|dept_no|
+------+----------+----------+---------+------+----------+-------+
| 12046|1961-05-16|     Xiong|    Ranum|     F|1996-01-13|   d001|
| 13289|1962-08-12|     Berni|     Baer|     F|1987-09-24|   d001|
| 14570|1963-07-26|    Chinho|     Bala|     F|1994-11-17|   d001|
| 18024|1952-08-04|       Uzi|Casperson|     M|1987-12-14|   d001|
| 23336|1958-10-07|     Kauko|     Suri|     F|1990-04-15|   d001|
+------+----------+----------+---------+------+----------+-------+
only showing top 5 rows



In [ ]:
# Department DF
df_department = df_departments.select("*")

df_department.show()

+-------+------------------+
|dept_no|         dept_name|
+-------+------------------+
|   d009|  Customer Service|
|   d005|       Development|
|   d010|     Entertainment|
|   d002|           Finance|
|   d003|   Human Resources|
|   d001|         Marketing|
|   d004|        Production|
|   d006|Quality Management|
|   d008|          Research|
|   d007|             Sales|
+-------+------------------+



Using those two new DataFrames, create and populate corresponding table in MySQL database employee and department. Report on the number of rows in each MySQL table.

In [ ]:
# Save employee DF to mysql
(
    df_employee.repartition(10).write
    .format("jdbc")
    .option("url","jdbc:mysql://127.0.0.1:3306")
    .option("driver","com.mysql.cj.jdbc.Driver")
    .option("dbtable", "employees.employee")
    .option("user", "root")
    .option("password", "password")
    .mode("overwrite")
    .save()
)

```bash
# employee table count in mysql:
mysql> select count(*) from employee;
+----------+
| count(*) |
+----------+
|   314257 |
+----------+
1 row in set (0.04 sec)
```

In [ ]:
# Save department DF to mysql:
(
    df_department.repartition(5).write
    .format("jdbc")
    .option("url","jdbc:mysql://127.0.0.1:3306")
    .option("driver","com.mysql.cj.jdbc.Driver")
    .option("dbtable", "employees.department")
    .option("user", "root")
    .option("password", "password")
    .mode("overwrite")
    .save()
)

```bash
# Show in MySql
mysql> select count(*) from department;
+----------+
| count(*) |
+----------+
|       10 |
+----------+
1 row in set (0.00 sec)
```

# Problem 3. 
On both MySQL side and on the Spark side find the number of employees in each department. Your results should include the name of each department. Please, select rows that have matching values in both tables. Use DataFrames employee and department and MySQL tables employee and department. On the Spark side, perform your query both by using DataFrame API and spark.SQL statements. Is your result different from the initial result obtained in Problem 1?

```bash
# Number of employees in each department [department, employee] tables - MySql
mysql> select count(e.emp_no) as total_emp, e.dept_no, d.dept_name from employee e
    -> join department d on d.dept_no = e.dept_no
    -> group by e.dept_no, d.dept_name;
+-----------+---------+--------------------+
| total_emp | dept_no | dept_name          |
+-----------+---------+--------------------+
|     85707 | d005    | Development        |
|     73485 | d004    | Production         |
|     52245 | d007    | Sales              |
|     20117 | d006    | Quality Management |
|     23580 | d009    | Customer Service   |
|     20211 | d001    | Marketing          |
|     17786 | d003    | Human Resources    |
|     21126 | d008    | Research           |
+-----------+---------+--------------------+
8 rows in set (1.20 sec)
```

Number of employees in each department - spark sql

In [ ]:
# temp view for employee
df_employee.createOrReplaceTempView("employee")

# temp view for department
df_department.createOrReplaceTempView("department")

In [ ]:
# Number of employees in each department
spark.sql(
    "select count(employee.emp_no) as total_emp_in_dept, employee.dept_no, department.dept_name"
    + " from employee join department on department.dept_no = employee.dept_no"
    + " group by employee.dept_no, department.dept_name"
).show()

+-----------------+-------+------------------+
|total_emp_in_dept|dept_no|         dept_name|
+-----------------+-------+------------------+
|            20211|   d001|         Marketing|
|            17786|   d003|   Human Resources|
|            73485|   d004|        Production|
|            85707|   d005|       Development|
|            20117|   d006|Quality Management|
|            52245|   d007|             Sales|
|            21126|   d008|          Research|
|            23580|   d009|  Customer Service|
+-----------------+-------+------------------+



Number of employees in each department -> DataFrame

In [ ]:
joinExp = df_employee["dept_no"] == df_department["dept_no"]

agg_count = (
    df_employee.alias("e")
    .join(df_department.alias("d"), joinExp)
    .select("e.emp_no", "e.dept_no", "d.dept_name")
    .groupBy("e.dept_no", "d.dept_name")
    .agg(count("e.emp_no").alias("total_emp_in_dept"))
)

agg_count.show()

+-------+------------------+-----------------+
|dept_no|         dept_name|total_emp_in_dept|
+-------+------------------+-----------------+
|   d001|         Marketing|            20211|
|   d003|   Human Resources|            17786|
|   d004|        Production|            73485|
|   d005|       Development|            85707|
|   d006|Quality Management|            20117|
|   d007|             Sales|            52245|
|   d008|          Research|            21126|
|   d009|  Customer Service|            23580|
+-------+------------------+-----------------+



Is your result different from the initial result obtained in Problem 1?
Ans: Yes -> 
     The results are different because we deleted the row for department d002 (Finance) <br>
     ***Note That the new data insterted is not included here becuase the default join is inner-join.***

# Problem 4. 
On both MySQL side and on the Spark side find the number of employees in each department. Your results should include the number of employees and the names of department where those employees work. Please include the names of departments which have no matching employees. On the Spark side perform your query both by using DataFrame API and spark.SQL statements.

```bash
# Count of employees in each department, empty departments inclusive - MySql
select count(e.emp_no) as total_emp, e.dept_no, d.dept_name from employee e
    -> right outer join department d on d.dept_no = e.dept_no
    -> group by e.dept_no, d.dept_name;
+-----------+---------+--------------------+
| total_emp | dept_no | dept_name          |
+-----------+---------+--------------------+
|     52245 | d007    | Sales              |
|     20211 | d001    | Marketing          |
|     17786 | d003    | Human Resources    |
|     73485 | d004    | Production         |
|     23580 | d009    | Customer Service   |
|     85707 | d005    | Development        |
|     20117 | d006    | Quality Management |
|     21126 | d008    | Research           |
|         0 | NULL    | Entertainment      |
|         0 | NULL    | Finance            |
+-----------+---------+--------------------+
10 rows in set (1.87 sec)
```

In [106]:
# Count of employees in each department, empty departments inclusive - spark.sql
spark.sql(
    "select count(employee.emp_no) as total_emp_in_dept, employee.dept_no, department.dept_name"
    + " from employee right outer join department on department.dept_no = employee.dept_no"
    + " group by employee.dept_no, department.dept_name"
).show()

+-----------------+-------+------------------+
|total_emp_in_dept|dept_no|         dept_name|
+-----------------+-------+------------------+
|                0|   null|     Entertainment|
|                0|   null|           Finance|
|            52245|   d007|             Sales|
|            73485|   d004|        Production|
|            17786|   d003|   Human Resources|
|            20211|   d001|         Marketing|
|            23580|   d009|  Customer Service|
|            21126|   d008|          Research|
|            85707|   d005|       Development|
|            20117|   d006|Quality Management|
+-----------------+-------+------------------+



In [107]:
# Count of employees in each department, empty departments inclusive - DataFrame
agg_count_empty = (
    df_employee.alias("e")
    .join(df_department.alias("d"), joinExp, "right_outer")
    .select("e.emp_no", "e.dept_no", "d.dept_name")
    .groupBy("e.dept_no", "d.dept_name")
    .agg(count("e.emp_no").alias("total_emp_in_dept"))
)

agg_count_empty.show()

+-------+------------------+-----------------+
|dept_no|         dept_name|total_emp_in_dept|
+-------+------------------+-----------------+
|   null|     Entertainment|                0|
|   null|           Finance|                0|
|   d007|             Sales|            52245|
|   d004|        Production|            73485|
|   d003|   Human Resources|            17786|
|   d001|         Marketing|            20211|
|   d009|  Customer Service|            23580|
|   d008|          Research|            21126|
|   d005|       Development|            85707|
|   d006|Quality Management|            20117|
+-------+------------------+-----------------+



# Problem 5. 
First on the Spark side and then on MySQL side, find the number of employees in each department. Your results should include the names of department(s) which have no matching employees and the number of employees without matching department, if any. On the Spark side perform your query both by using DataFrame API and spark.SQL statements. Your results should include the names of department(s) which have no matching employees and the number of employees without matching department, if any. This problem might require some independent reading due to implementation limitations on MySQL side. If having issues, just report your findings.

First on the Spark side and then on MySQL side, find the number of employees in each department. Your results should include the names of department(s) which have no matching employees and the number of employees without matching department, if any.

In [120]:
# Number of employees in each department(s), empty departments inclusive - Spark DF

# Join Expressions
joinExpEmp = df_employees["emp_no"] == df_dept_emp["emp_no"]
joinExpDeptEmp = df_departments["dept_no"] == df_dept_emp["dept_no"]

# Using department(s) and employee(s) tables 
(
    df_employees.alias("e")
    .join(df_dept_emp.alias("d"), joinExpEmp, "right_outer")
    .join(df_departments.alias("de"), joinExpDeptEmp, "right_outer")
    .select("e.emp_no", "de.dept_no", "de.dept_name")
    .groupBy("de.dept_no", "de.dept_name")
    .agg(count("e.emp_no").alias("total_emp_in_dept"))
    .show()
)

+-------+------------------+-----------------+
|dept_no|         dept_name|total_emp_in_dept|
+-------+------------------+-----------------+
|   d005|       Development|            85707|
|   d009|  Customer Service|            23580|
|   d003|   Human Resources|            17786|
|   d001|         Marketing|            20211|
|   d007|             Sales|            52245|
|   d004|        Production|            73485|
|   d010|     Entertainment|                0|
|   d002|           Finance|                0|
|   d006|Quality Management|            20117|
|   d008|          Research|            21126|
+-------+------------------+-----------------+



In [123]:
# temp view for employee
df_employees.createOrReplaceTempView("employees")

# temp view for department
df_departments.createOrReplaceTempView("departments")

# temp view for dept_emp
df_dept_emp.createOrReplaceTempView("dept_emp")

In [130]:
spark.sql(
    "select count(employees.emp_no) as total_emp_in_dept, dept_emp.dept_no, departments.dept_name"
    + " from employees" 
    + " right outer join dept_emp on dept_emp.emp_no = employees.emp_no"
    + " right outer join departments on dept_emp.dept_no = departments.dept_no"
    + " group by dept_emp.dept_no, departments.dept_name"
).show()

+-----------------+-------+------------------+
|total_emp_in_dept|dept_no|         dept_name|
+-----------------+-------+------------------+
|                0|   null|     Entertainment|
|                0|   null|           Finance|
|            52245|   d007|             Sales|
|            73485|   d004|        Production|
|            17786|   d003|   Human Resources|
|            20211|   d001|         Marketing|
|            23580|   d009|  Customer Service|
|            21126|   d008|          Research|
|            85707|   d005|       Development|
|            20117|   d006|Quality Management|
+-----------------+-------+------------------+



Number of employees that don't have a department

In [133]:
# Dataframe
(
    df_employees.alias("e")
    .join(df_dept_emp.alias("d"), joinExpEmp, "left_anti")
    .count()
)

14718

In [138]:
# spark.sql

spark.sql("select count(*) from employees "
    + "left anti join dept_emp on employees.emp_no = dept_emp.emp_no").show()

+--------+
|count(1)|
+--------+
|   14718|
+--------+



Employees with no department - MySql

```bash
select count(*) from employees e
    -> where not exists (select * from dept_emp d where e.emp_no = d.emp_no);
+----------+
| count(*) |
+----------+
|    14718 |
+----------+
1 row in set (1.82 sec)
```